In [ ]:
%pip install gadm hdx-python-api
%pip install geopandas --upgrade
%pip install pyomo
%pip install highspy
%pip install chart_studio
%pip install h5netcdf

In [3]:
import folium as fl
import pandas as pd
import geopandas as gpd
from hdx.api.configuration import Configuration
from hdx.data.resource import Resource
import urllib.request
import requests
import json
import requests
import itertools

from shapely.geometry import Polygon,MultiPolygon
from shapely.ops import unary_union

from gadm import GADMDownloader
import numpy as np

import xarray as xr
import plotly.graph_objects as go
import chart_studio
import chart_studio.plotly as py

import matplotlib.pyplot as plt

In [4]:
# Initialize the GADMDownloader with the specified version (in this case, version 4.0)
downloader = GADMDownloader(version="4.0")

# Define the country name for which you want to retrieve administrative boundary data
country_name = "Serbia"

# Specify the administrative level you are interested in (e.g., 1 for districts or provinces)
ad_level = 0

# Retrieve the geospatial data for the selected country and administrative level
gdf = downloader.get_shape_data_by_country_name(country_name=country_name, ad_level=ad_level)

# Display the first 2 rows of the obtained geospatial data for a quick preview
gdf.head(2)


,COUNTRY,ID_0,geometry
0,Serbia,SRB,"MULTIPOLYGON (((22.12066 42.30355, 22.10080 42..."


In [5]:
# Create a Folium map (m) with an initial zoom level of 10 and using OpenStreetMap tiles as the basemap
m = fl.Map(location=[44.0165, 21.0059], zoom_start=8, tiles="OpenStreetMap")

# Iterate through each row in the geospatial data (gdf) representing administrative boundaries
for _, r in gdf.iterrows():
    # Simplify the geometry of the current boundary with a specified tolerance
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.00001)

    # Convert the simplified geometry to JSON format
    geo_j = sim_geo.to_json()

    # Create a GeoJson layer from the JSON geometry, and style it with an orange fill color
    geo_j = fl.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "orange"})

    # Add a popup with the NAME_1 attribute (administrative region name) to the GeoJson layer
    fl.Popup(r["COUNTRY"]).add_to(geo_j)

    # Add the styled GeoJson layer to the Folium map (m)
    geo_j.add_to(m)

# Display the Folium map (m) with the administrative boundaries and popups
#m


In [6]:
selected_gadm = gdf

In [7]:
population = pd.read_csv('ppp_SRB_2020_1km_Aggregated.csv')

In [8]:
print('Total Population:',round(population['Z'].sum()/1000000,2),'million')
population = gpd.GeoDataFrame(
    population, geometry=gpd.points_from_xy(population.X, population.Y)
)

population = population.set_crs(selected_gadm.crs)

Total Population: 6.98 million


In [9]:
# Perform a spatial join to find population within the selected administrative boundary
population_aoi = gpd.sjoin(population, selected_gadm)[['X','Y','Z','geometry']]
population_aoi.columns = ['Longitude','Latitude','Population','geometry']
population_aoi = population_aoi.reset_index().reset_index()
del population_aoi['index']
population_aoi = population_aoi.rename(columns={'level_0':'ID'})
print('Total Population:',round(population_aoi['Population'].sum()))

Total Population: 6968863


### This code segment retrieves and analyzes healthcare facility data (hospitals and clinics) in Serbia within a specified area of interest (AOI). Here's a brief summary of what it does:

- It uses the Overpass API to query OpenStreetMap data for hospitals in Timor-Leste, retrieves the data in JSON format, and converts it into a DataFrame (`df_hospitals`).

- It extracts relevant information, such as the hospital's name, latitude, and longitude, from the OpenStreetMap data.

- Similarly, it queries OpenStreetMap data for clinics in Timor-Leste, retrieves the data, and processes it into a DataFrame (`df_clinics`), extracting relevant information.

- The code then combines the hospital and clinic data into a single GeoDataFrame (`df_health_osm`) and converts latitude and longitude coordinates into a geometry column.

- It prints the number of hospitals and clinics extracted from the data.

- Finally, it performs a spatial join to determine how many hospitals and clinics fall within the specified administrative region of interest (AOI) and prints the result.

This code segment is a critical step in assessing healthcare accessibility in a specific region of Timor-Leste, as it identifies and quantifies the healthcare facilities within the chosen area.

In [10]:
%%time

overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["ISO3166-1"="RS"];
(node["amenity"="hospital"](area);
 way["amenity"="hospital"](area);
 rel["amenity"="hospital"](area);
);
out center;
"""
response = requests.get(overpass_url,
                        params={'data': overpass_query})
data = response.json()

df_hospitals = pd.DataFrame(data['elements'])

df_hospitals['name'] = df_hospitals['tags'].apply(lambda x:x['name'] if 'name' in list(x.keys()) else None)

df_hospitals = df_hospitals[['id','lat','lon','name']].drop_duplicates()

overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["ISO3166-1"="RS"];
(node["amenity"="clinic"](area);
 way["amenity"="clinic"](area);
 rel["amenity"="clinic"](area);
);
out center;
"""
response = requests.get(overpass_url,
                        params={'data': overpass_query})
data = response.json()

df_clinics = pd.DataFrame(data['elements'])
df_clinics['name'] = df_clinics['tags'].apply(lambda x:x['name'] if 'name' in list(x.keys()) else None)
df_clinics['amenity'] = df_clinics['tags'].apply(lambda x:x['healthcare'] if 'healthcare' in list(x.keys()) else None)

df_clinics = df_clinics[['id','lat','lon','name','amenity']].drop_duplicates()

df_health_osm = pd.concat([df_hospitals,df_clinics])
df_health_osm = gpd.GeoDataFrame(df_health_osm, geometry=gpd.points_from_xy(df_health_osm.lon, df_health_osm.lat))
df_health_osm = df_health_osm[['id','name','geometry']]

print('Number of hospitals and clinics extracted:',len(df_health_osm))
df_health_osm = df_health_osm.set_crs(selected_gadm.crs)
selected_hosp = gpd.sjoin(df_health_osm, selected_gadm, predicate='within')
print('Number of hospitals and clinics in Serbia',len(selected_hosp))

Number of hospitals and clinics extracted: 569
Number of hospitals and clinics in Serbia 233
CPU times: total: 46.9 ms
Wall time: 2.68 s


In [11]:
len(selected_hosp)

233

In [12]:
selected_hosp.head()

,id,name,geometry,index_right,COUNTRY,ID_0
0,291178034,Здравствена станица „Авијатичарски трг“,POINT (20.41386 44.83980),0,Serbia,SRB
2,461185545,Дом здравља,POINT (22.07562 42.89118),0,Serbia,SRB
6,708953669,None,POINT (22.26338 43.90570),0,Serbia,SRB
7,779174314,Црвени крст,POINT (22.28202 43.89763),0,Serbia,SRB
8,835423002,Здравствена амбуланта,POINT (22.52470 44.29361),0,Serbia,SRB


In [13]:
def get_isochrone_osm (each_hosp,travel_time_secs):
  body = {"locations":[[each_hosp.x,each_hosp.y]],"range":[travel_time_secs],"range_type":'time'}
  headers = {
      'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
      'Authorization': '5b3ce3597851110001cf62487e439dfd168049a694bf909262583588',
      'Content-Type': 'application/json; charset=utf-8'
  }
  call = requests.post('https://api.openrouteservice.org/v2/isochrones/driving-car', json=body, headers=headers)
  if(call.status_code==200):
    geom = (json.loads(call.text)['features'][0]['geometry'])
    polygon_geom = Polygon(geom['coordinates'][0])
    return polygon_geom
  else:
    return None

In [14]:
# Generating travel times in seconds from 10 minutes to 120 minutes, incremented by 10 minutes
travel_times = [x * 60 for x in range(10, 121, 10)]
print("Travel times in seconds:", travel_times)

# Create a small subset of the dataframe for testing
test_subset = selected_hosp.head(5).copy()

# Applying the get_isochrone_osm function for each travel time and storing the results
for time in travel_times:
    column_name = f'catchment_area_osm_{time}'
    test_subset[column_name] = test_subset['geometry'].apply(lambda x: get_isochrone_osm(x, travel_time_secs=time))

# Print the results for each travel time
for time in travel_times:
    column_name = f'catchment_area_osm_{time}'
    print(f"Results for travel time {time} seconds:")
    print(test_subset[['id', column_name]])

Travel times in seconds: [600, 1200, 1800, 2400, 3000, 3600, 4200, 4800, 5400, 6000, 6600, 7200]
Results for travel time 600 seconds:
          id                             catchment_area_osm_600
0  291178034  POLYGON ((20.32537 44.83279, 20.32880 44.83273...
2  461185545  POLYGON ((22.02066 42.92048, 22.02616 42.91855...
6  708953669  POLYGON ((22.19716 43.90392, 22.19830 43.89679...
7  779174314  POLYGON ((22.21191 43.90077, 22.21237 43.90049...
8  835423002  POLYGON ((22.47368 44.34206, 22.47363 44.34180...
Results for travel time 1200 seconds:
          id                            catchment_area_osm_1200
0  291178034  POLYGON ((20.12060 44.87704, 20.12285 44.87593...
2  461185545  POLYGON ((21.91953 43.11345, 21.92171 43.11164...
6  708953669  POLYGON ((22.15638 44.01074, 22.15795 44.00932...
7  779174314  POLYGON ((22.16990 43.99840, 22.17067 43.99805...
8  835423002  POLYGON ((22.45430 44.40367, 22.45441 44.40208...
Results for travel time 1800 seconds:
          id          

In [ ]:
quartile_labels = [0.1, 0.25, 0.5, 1.0]
population_aoi['opacity'] = pd.qcut(population_aoi['Population'], 4, labels=quartile_labels)

In [ ]:
def get_pop_count(cachment,pop_data):
  if(cachment!=None):
    pop_access = pop_data[pop_data.within(cachment)]
    id_values = (pop_access['ID'].values)
    pop_with_access = (pop_access['Population'].sum().round())
    return id_values,pop_with_access
  else:
    return [None,None]

selected_hosp['id_with_access'], selected_hosp['pop_with_access'] = zip(*selected_hosp['cachment_area'].apply(get_pop_count, pop_data=population_aoi))


In [ ]:
selected_hosp = selected_hosp.dropna()

In [ ]:
list_ids_access = [ids if ids is not None else [] for ids in selected_hosp['id_with_access']]
list_ids_access = list(itertools.chain.from_iterable(list_ids_access))
pop_with_access = population_aoi[population_aoi['ID'].isin(list_ids_access)]
pop_without_access = population_aoi[~population_aoi['ID'].isin(list_ids_access)]

original_access = round(pop_with_access['Population'].sum()*100/population_aoi['Population'].sum(),2)

print('Population with Access:',round(pop_with_access['Population'].sum()*100/population_aoi['Population'].sum(),2),'%')

In [ ]:
# Calculate the population with access for each travel time
results = calculate_population_with_access(travel_times, selected_hosp, population_aoi)

# Create a DataFrame to store the results
results_df = pd.DataFrame(results, columns=['Travel Time (minutes)', 'Population with Access (%)'])
print(results_df)

In [ ]:
# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(results_df['Travel Time (minutes)'], results_df['Population with Access (%)'], marker='o')
plt.title('Travel Time vs Population with Access')
plt.xlabel('Travel Time (minutes)')
plt.ylabel('Population with Access (%)')
plt.grid(True)
plt.show()
